In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [6]:
class rnn (nn.Module) :

    def __init__(self,hidden_size, input_size, output_size):
        super(rnn, self).__init__()

        self.hidden_size = hidden_size
        self.i2h = nn.Linear(hidden_size+input_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size , output_size)
        self.softmax = nn.LogSoftmax(dim = 1)

    def forward(self, input, hidden):
        combined = torch.cat((input,hidden),1)
        hidden = self.i2h(combined)
        output = self.i2o(combined)
        output = self.softmax(output)
        
        return output, hidden

    def init_hidden(self):

        return torch.zeros(1, self.hidden_size)

In [7]:
n_samples = 1000
input_size = 15
output_size = 2
hidden_size = 20 


# Generate random data
X = torch.randn(n_samples, input_size)
y = torch.randint(0, output_size, (n_samples,))

train_dataset = TensorDataset(X, y)
train_loader = DataLoader(train_dataset, batch_size=5, shuffle=True)

In [11]:
model = rnn(hidden_size, input_size, output_size)
criterion = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Training loop
n_epochs = 100

for epoch in range(n_epochs):
    for x_batch, y_batch in train_loader:
        hidden = model.init_hidden().repeat(x_batch.size(0), 1)  

        optimizer.zero_grad()
        output, hidden = model(x_batch, hidden)  

        loss = criterion(output, y_batch)  
        loss.backward()
        optimizer.step()

    if epoch % 10 == 0:
        print(f'Epoch: {epoch}  Loss: {loss.item()}')

Epoch: 0  Loss: 0.7219048738479614
Epoch: 10  Loss: 0.6429959535598755
Epoch: 20  Loss: 0.7288631796836853
Epoch: 30  Loss: 0.6572973132133484
Epoch: 40  Loss: 0.7310658693313599
Epoch: 50  Loss: 0.6076589822769165
Epoch: 60  Loss: 0.8018785715103149
Epoch: 70  Loss: 0.6949945092201233
Epoch: 80  Loss: 0.7895050048828125
Epoch: 90  Loss: 0.5974259376525879
